In [119]:
import pandas as pd
import numpy as np
import json
import re
import nltk

from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Read data file into a python array
with open('../data/test_tip.json', 'rb') as f:
    bus_data = f.readlines()

# remove the trailing "\n" from each line
bus_data = map(lambda x: x.rstrip(), bus_data)
# put individual business JSON objects into list
data_json = "[" + ','.join(bus_data) + "]"

# Create pandas df
bus_df = pd.read_json(data_json)

In [125]:
bus_df

,business_id,date,likes,text,type,user_id
0,cE27W9VPgO88Qxe4ol6y_g,2013-04-18,0,Don't waste your time.,tip,-6rEfobYjMxpUWLNxszaxQ
1,mVHrayjG3uZ_RLHkLj-AMg,2013-01-06,1,Your GPS will not allow you to find this place...,tip,EZ0r9dKKtEGVx2CdnowPCw
2,KayYbHCt-RkbGcPdGOThNg,2013-12-03,0,Great drink specials!,tip,xb6zEQCw9I-Gl0g06e1KsQ
3,KayYbHCt-RkbGcPdGOThNg,2015-07-08,0,"Friendly staff, good food, great beer selectio...",tip,QawZN4PSW7ng_9SP7pjsVQ
4,1_lU0-eSWJCRvNGk78Zh9Q,2015-10-25,0,Beautiful restoration.,tip,MLQre1nvUtW-RqMTc4iC9A
5,1_lU0-eSWJCRvNGk78Zh9Q,2015-01-06,0,Home to Stage 62 theatre group.,tip,bvu13GyOUwhEjPum2xjiqQ
6,_qopVQ6_Mz6W7-Pmbi56GQ,2013-02-13,0,A God send if you're not a gear head!,tip,bvu13GyOUwhEjPum2xjiqQ
7,_qopVQ6_Mz6W7-Pmbi56GQ,2010-08-27,0,Great people ... great service ... always busy,tip,_QFom7aSHKNCDsNXKd-3xQ
8,wJr6kSA5dchdgOdwH6dZ2w,2013-07-22,0,Sarah rocks! Best waitress here! Be sure to ge...,tip,fvTivrsJoUMYXnOJw9wZfw
9,Cdcus0NADzyY3XiJM2O5Sg,2011-10-12,0,Unleaded 3.42,tip,bvu13GyOUwhEjPum2xjiqQ


In [210]:
# Helper functions for normalising text data

# Convert all words to lowercase, remove punctuation, tokenise and stem
# and remove stopwords, threshold = 10%
def norm_corpus(document_list):
    norm_doc_list = []
    
    # lowercase
    document_list = [word.lower() for word in document_list]

    
    # remove symbols in text
    symbols = ",.?!"
    for sym in symbols:
        document_list = [word.replace(sym,'') for word in document_list]
    
    
    # loop through each string i.e. review in the column
    for doc in document_list:
        doc = nltk.word_tokenize(doc)
        
        # remove stopwords
        doc = [word for word in doc if word not in stopwords.words('english')]
        
        # stem words
        stemmer = SnowballStemmer("english")
        doc = [stemmer.stem(word) for word in doc]
        
        # make tokenised text one string
        norm_doc = " ".join(doc)
        norm_doc_list.append(norm_doc)
    
    return norm_doc_list

bus_df['tip'] = norm_corpus(bus_df.text)
bus_df

,business_id,date,likes,text,type,user_id,tip
0,cE27W9VPgO88Qxe4ol6y_g,2013-04-18,0,Don't waste your time.,tip,-6rEfobYjMxpUWLNxszaxQ,"[, , n, ', , , w, , , , e, , , , u, r, , , , ,..."
1,mVHrayjG3uZ_RLHkLj-AMg,2013-01-06,1,Your GPS will not allow you to find this place...,tip,EZ0r9dKKtEGVx2CdnowPCw,"[, , u, r, , g, p, , , w, , l, l, , n, , , , ,..."
2,KayYbHCt-RkbGcPdGOThNg,2013-12-03,0,Great drink specials!,tip,xb6zEQCw9I-Gl0g06e1KsQ,"[g, r, e, , , , , r, , n, k, , , p, e, c, , , ..."
3,KayYbHCt-RkbGcPdGOThNg,2015-07-08,0,"Friendly staff, good food, great beer selectio...",tip,QawZN4PSW7ng_9SP7pjsVQ,"[f, r, , e, n, , l, , , , , , f, f, , , g, , ,..."
4,1_lU0-eSWJCRvNGk78Zh9Q,2015-10-25,0,Beautiful restoration.,tip,MLQre1nvUtW-RqMTc4iC9A,"[b, e, , u, , , f, u, l, , r, e, , , , r, , , ..."
5,1_lU0-eSWJCRvNGk78Zh9Q,2015-01-06,0,Home to Stage 62 theatre group.,tip,bvu13GyOUwhEjPum2xjiqQ,"[h, , , e, , , , , , , , g, e, , 6, 2, , , h, ..."
6,_qopVQ6_Mz6W7-Pmbi56GQ,2013-02-13,0,A God send if you're not a gear head!,tip,bvu13GyOUwhEjPum2xjiqQ,"[, , g, , , , , e, n, , , , f, , , , u, ', r, ..."
7,_qopVQ6_Mz6W7-Pmbi56GQ,2010-08-27,0,Great people ... great service ... always busy,tip,_QFom7aSHKNCDsNXKd-3xQ,"[g, r, e, , , , p, e, , p, l, e, , , , , , g, ..."
8,wJr6kSA5dchdgOdwH6dZ2w,2013-07-22,0,Sarah rocks! Best waitress here! Be sure to ge...,tip,fvTivrsJoUMYXnOJw9wZfw,"[, , r, , h, , r, , c, k, , , , b, e, , , , w,..."
9,Cdcus0NADzyY3XiJM2O5Sg,2011-10-12,0,Unleaded 3.42,tip,bvu13GyOUwhEjPum2xjiqQ,"[u, n, l, e, , , e, , , 3, , 4, 2]"


In [ ]:
# Helper functions for vectorise normalised data

In [197]:
# Vectorise keywords from normalised text to vector including only nouns and adjectives
def review_vector(norm_doc_list):
    review_list = []

    # select all words categorised as nouns or adjectives
    # loop through each string i.e. review in the df column
    for doc in norm_doc_list:
        review_keyword_list = []
        doc = nltk.word_tokenize(doc)
        # create tuple for each word in list: (word, tag)
        token_category = nltk.pos_tag(doc)
        
        
        for word, tag in token_category:    
            
            # nouns
            if (tag == 'NN' or tag == 'NNS' or tag == 'NNP' or tag == 'NNPS'):
                review_keyword_list.append(word)
                
            # adjectives
            elif (tag == 'JJ' or tag == 'JJS' or tag == 'JJP' or tag == 'JJPS'):
                review_keyword_list.append(word)
            else:
                pass     
        
        review_keywords = " ".join(review_keyword_list)
        review_list.append(review_keywords)
        
        # vectorise string
        WORD = re.compile(r'\w+')
        review_vector = [collections.Counter(WORD.findall(word)) for word in review_list]
    
    
    return review_vector
    
# test = norm_corpus(bus_df.text)
review_vector(test)

[Counter({u'time': 1, u'wast': 1}),
 Counter({u'depart': 1,
          u'direct': 1,
          u'find': 1,
          u'gps': 1,
          u'place': 1,
          u'polic': 1,
          u'rankin': 1,
          u'street': 1}),
 Counter({u'great': 1, u'special': 1}),
 Counter({u'beer': 1,
          u'food': 1,
          u'friend': 1,
          u'good': 1,
          u'great': 1,
          u'select': 1,
          u'staff': 1}),
 Counter({u'beauti': 1, u'restor': 1}),
 Counter({u'group': 1, u'home': 1, u'stage': 1, u'theatr': 1}),
 Counter({u'god': 1, u'head': 1, u're': 1, u'send': 1}),
 Counter({u'alway': 1, u'busi': 1, u'great': 2, u'peopl': 1, u'servic': 1}),
 Counter({u'best': 1,
          u'compliment': 1,
          u'glass': 1,
          u'rock': 1,
          u'sarah': 1,
          u'sure': 1,
          u'waitress': 1}),
 Counter({u'unlead': 1})]

In [217]:
output_df = bus_df[['business_id', 'user_id', 'date']]  
output_df['tip'] = norm_corpus(bus_df.text)
output_df.tip = review_vector(output_df.tip)
output_df

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,business_id,user_id,date,tip
0,cE27W9VPgO88Qxe4ol6y_g,-6rEfobYjMxpUWLNxszaxQ,2013-04-18,"{u'wast': 1, u'time': 1}"
1,mVHrayjG3uZ_RLHkLj-AMg,EZ0r9dKKtEGVx2CdnowPCw,2013-01-06,"{u'depart': 1, u'polic': 1, u'rankin': 1, u'di..."
2,KayYbHCt-RkbGcPdGOThNg,xb6zEQCw9I-Gl0g06e1KsQ,2013-12-03,"{u'great': 1, u'special': 1}"
3,KayYbHCt-RkbGcPdGOThNg,QawZN4PSW7ng_9SP7pjsVQ,2015-07-08,"{u'great': 1, u'good': 1, u'food': 1, u'beer':..."
4,1_lU0-eSWJCRvNGk78Zh9Q,MLQre1nvUtW-RqMTc4iC9A,2015-10-25,"{u'beauti': 1, u'restor': 1}"
5,1_lU0-eSWJCRvNGk78Zh9Q,bvu13GyOUwhEjPum2xjiqQ,2015-01-06,"{u'home': 1, u'theatr': 1, u'group': 1, u'stag..."
6,_qopVQ6_Mz6W7-Pmbi56GQ,bvu13GyOUwhEjPum2xjiqQ,2013-02-13,"{u'god': 1, u'head': 1, u'send': 1, u're': 1}"
7,_qopVQ6_Mz6W7-Pmbi56GQ,_QFom7aSHKNCDsNXKd-3xQ,2010-08-27,"{u'peopl': 1, u'great': 2, u'busi': 1, u'servi..."
8,wJr6kSA5dchdgOdwH6dZ2w,fvTivrsJoUMYXnOJw9wZfw,2013-07-22,"{u'sarah': 1, u'sure': 1, u'best': 1, u'glass'..."
9,Cdcus0NADzyY3XiJM2O5Sg,bvu13GyOUwhEjPum2xjiqQ,2011-10-12,{u'unlead': 1}
